###Produce constructor standings

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

Find race years for which the data is to be reprocessed

In [0]:
race_results_list = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
.filter(f"file_date = '{v_file_date}'") \
.select("race_year") \
.distinct() \
.collect()

In [0]:
race_year_list = []
for race_year in race_results_list:
    race_year_list.append(race_year.race_year)

In [0]:
from pyspark.sql.functions import col

race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
.filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, when, count, col

constructor_standings_df = race_results_df \
    .groupBy("race_year","team") \
    .agg(sum("points").alias("total_points"),
         count(when(col("position") == 1,True)).alias("wins"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standings_df.withColumn("rank",rank().over(constructor_rank_spec))

In [0]:
merge_condition = "tgt.race_year = src.race_year AND tgt.team = src.team"
merge_delta_data(final_df, "f1_presentation", "constructor_standings", presentation_folder_path, merge_condition, "race_year")